In [1]:
import os
import time
import stripe

#from utils.custom_exceptions import RaiseCustomException

In [2]:
stripe.api_key = "sk_test_51O21ZuEIx0QzrdrDkRpaZAixfOxeamUkUp18LW2IxRorplnTTjC2Fd7lMZHwqBVNMKkl3pV6cPFR4Wuq4KGS34Fm003470GztC"

### Create stripe customer

##### Function definition

In [3]:
def create_stripe_customer(email=None, name=None, description=None, metadata=None):
    """
    :param email: str - Email address of the customer.
    :param name: str - Full name of the customer.
    :param description: str - Optional description for the customer.
    :param metadata: dict - Optional metadata for the customer.
    :return: dict - Details of the created customer or None if there was an error.
    """
    try:
        customer = stripe.Customer.create(email=email, name=name, description=description, metadata=metadata)
        return customer["id"]
    except stripe.error.StripeError as e:
        raise Exception(f"Failed to create stripe customer: {e}")


##### Function call

In [4]:
customer_id = create_stripe_customer(email="k2@k.com", name="k m", description="Test customer", metadata={"user_id": "docudive id"})
customer_id

'cus_OqFDzaQXQdrkAn'

### Create subscription for customer

##### Function definition

In [3]:
customer_id = "cus_OqIRXeiTNJWyvD"

In [4]:
def create_subscription(customer_id, price_id):
    """
    Create a subscription for a customer based on a price.
    :param customer_id: The Stripe ID of the customer.
    :param price_id: The Stripe ID of the price associated with the product. Displayed in the product details page
    :return: The subscription object if successful, otherwise an error.
    """
    try:
        subscription = stripe.Subscription.create(
            customer=customer_id,
            items=[{'price': price_id}],
            expand=['latest_invoice.payment_intent']
        )
        return subscription #this subscription item id should be stored in DB. This will be used in report_usage() function
    except stripe.error.StripeError as e:
        raise Exception(f"Failed to create subscription: {e}")

##### Function call

In [8]:
price_id = "price_1O21dpEIx0QzrdrDOQ38JaPc"
subscription_data = create_subscription(customer_id, price_id)
subscription_item_id = subscription_data["items"]["data"][0].id
subscription_item_id

'si_OrHTcU2wjBtOrJ'

### Report daily usage

##### Function definition

In [9]:
subscription_item_id = "si_OrHTcU2wjBtOrJ"

In [10]:
def report_usage(subscription_item_id, quantity, timestamp=None): #This should be fired everyday
    """
    Report usage for a metered billing subscription.
    :param subscription_item_id: str - The ID of the subscription item.
    :param quantity: int - The quantity/amount of usage to report.
    :param timestamp: int (optional) - The timestamp at which the usage occurred (in Unix time). Defaults to current time.
    :return: dict - Details of the created usage record.
    """
    try:
        usage_record = stripe.UsageRecord.create(
            subscription_item=subscription_item_id,
            quantity=quantity,
            timestamp=timestamp or int(time.time())
        )
        return usage_record
    except stripe.error.StripeError as e:
        raise Exception(f"Failed to report usage: {e}")

##### Function call

In [12]:
quantity = 1000  # The actual usage amount, e.g., credit point.
report_usage(subscription_item_id, quantity)

<UsageRecord usage_record id=mbur_1O3YuzEIx0QzrdrDwQGURoLP at 0x7fb28070eed0> JSON: {
  "id": "mbur_1O3YuzEIx0QzrdrDwQGURoLP",
  "livemode": false,
  "object": "usage_record",
  "quantity": 1000,
  "subscription_item": "si_OrHTcU2wjBtOrJ",
  "timestamp": 1697870657
}

### Billing history

##### Function definition

In [15]:
from datetime import datetime
def get_invoices_for_customer(customer_id, limit=12):
    """
    Retrieve a list of invoices for a specific customer with required details.
    :param customer_id: str - The Stripe customer ID.
    :param limit: int - The number of records to retrieve (maximum is 100).
    :return: list - List of invoices with specified details.
    """
    try:
        invoices = stripe.Invoice.list(customer=customer_id, limit=limit)
    except stripe.error.StripeError as e:
        raise Exception(f"Failed to retrieve invoices: {e}")

    invoice_details = []

    for invoice in invoices:
        # Converting the Unix timestamp to a datetime object
        dt_object = datetime.fromtimestamp(invoice.created)

        detail = {
            'invoice_number': invoice.number,
            'amount_due': invoice.amount_due / 100.0,
            'currency': invoice.currency,
            'status': invoice.status,
            'created_date': dt_object.isoformat(),
            'pdf_link': invoice.invoice_pdf
        }
        invoice_details.append(detail)

    return invoice_details


##### Function call

In [16]:
billing_history = get_invoices_for_customer(customer_id, limit=12)
billing_history

[{'invoice_number': 'F8E8E635-0004',
  'amount_due': 0.0,
  'currency': 'usd',
  'status': 'paid',
  'created_date': '2023-10-21T12:13:59',
  'pdf_link': 'https://pay.stripe.com/invoice/acct_1O21ZuEIx0QzrdrD/test_YWNjdF8xTzIxWnVFSXgwUXpyZHJELF9PckhUTlk5SHc5WXp0bDFyRnYweHpxQVFMM0VDUHZpLDg4NDExNTg20200f6pXtwZL/pdf?s=ap'},
 {'invoice_number': 'F8E8E635-0003',
  'amount_due': 0.0,
  'currency': 'usd',
  'status': 'paid',
  'created_date': '2023-10-18T21:21:05',
  'pdf_link': 'https://pay.stripe.com/invoice/acct_1O21ZuEIx0QzrdrD/test_YWNjdF8xTzIxWnVFSXgwUXpyZHJELF9PcUljaWRpYmRSdXl6RzBuMkZmNXdZM2ZpZUFKS1hRLDg4NDExNTg2020055hAsRBu/pdf?s=ap'},
 {'invoice_number': 'F8E8E635-0002',
  'amount_due': 0.0,
  'currency': 'usd',
  'status': 'paid',
  'created_date': '2023-10-18T21:21:03',
  'pdf_link': 'https://pay.stripe.com/invoice/acct_1O21ZuEIx0QzrdrD/test_YWNjdF8xTzIxWnVFSXgwUXpyZHJELF9PcUljZGVJdjdIWHptVUkwRTNyZTBmVVh2dFJUN3gzLDg4NDExNTg202008obQZPoU/pdf?s=ap'},
 {'invoice_number': 'F8E8E635-0001

In [20]:
billing_history

[{'invoice_number': 'F8E8E635-0001',
  'amount_due': 0.0,
  'currency': 'usd',
  'status': 'paid',
  'created_date': 1697643862,
  'pdf_link': 'https://pay.stripe.com/invoice/acct_1O21ZuEIx0QzrdrD/test_YWNjdF8xTzIxWnVFSXgwUXpyZHJELF9PcUlXRGx4ZlhDTHJNaW5TcE5VelVIN3ZYd0x5eWdwLDg4MTg1MDIz02006V9WA5Nw/pdf?s=ap'}]

### Billing preferences

##### Function definition

In [12]:
def update_customer_billing_details(customer_id, billing_details):
    """
    Update the billing details for a specific Stripe customer.
    :param customer_id: str - The Stripe customer ID.
    :param billing_details: dict - A dictionary containing billing details like name, address, email, etc.
    :return: dict - Updated customer object or None if there was an error.
    """
    try:
        customer = stripe.Customer.modify(
            customer_id,
            name=billing_details.get("name"),
            email=billing_details.get("email"),
            phone=billing_details.get("phone"),
            address={
                "line1": billing_details.get("line1"),
                "line2": billing_details.get("line2"),
                "city": billing_details.get("city"),
                "state": billing_details.get("state"),
                "postal_code": billing_details.get("postal_code"),
                "country": billing_details.get("country")
            }
        )
        return customer
    except stripe.error.StripeError as e:
        raise Exception(f"Failed to update billing details: {e}")

##### Function call

In [13]:
billing_info = {
    "name": "John Doe",
    "email": "john@example.com",
    "phone": "+1234567890",
    "line1": "123 Street Ave",
    "line2": "Apt 4B",
    "city": "City",
    "state": "State",
    "postal_code": "12345",
    "country": "US"
}

updated_customer = update_customer_billing_details(customer_id, billing_info) 
len(updated_customer)

22

In [14]:
updated_customer

<Customer customer id=cus_OqFDzaQXQdrkAn at 0x7f44d12d33d0> JSON: {
  "address": {
    "city": "City",
    "country": "US",
    "line1": "123 Street Ave",
    "line2": "Apt 4B",
    "postal_code": "12345",
    "state": "State"
  },
  "balance": 0,
  "created": 1697631598,
  "currency": "usd",
  "default_source": null,
  "delinquent": false,
  "description": "Test customer",
  "discount": null,
  "email": "john@example.com",
  "id": "cus_OqFDzaQXQdrkAn",
  "invoice_prefix": "F86DB4B3",
  "invoice_settings": {
    "custom_fields": null,
    "default_payment_method": null,
    "footer": null,
    "rendering_options": null
  },
  "livemode": false,
  "metadata": {
    "user_id": "docudive id"
  },
  "name": "John Doe",
  "next_invoice_sequence": 2,
  "object": "customer",
  "phone": "+1234567890",
  "preferred_locales": [],
  "shipping": null,
  "tax_exempt": "none",
  "test_clock": null
}

### Payment methods - Retrive

##### Function definition

In [77]:
def retrieve_all_payment_methods(customer_id):
    """
    Retrieve all payment methods associated with a specific Stripe customer.
    :param customer_id: str - ID of the Stripe customer.
    :return: list - List of payment methods or None if there was an error.
    """
    try:
        payment_methods = stripe.PaymentMethod.list(customer=customer_id).get("data")
        return payment_methods
    except stripe.error.StripeError as e:
        raise Exception(f"Failed to retrieve payment methods: {e}")

##### Function call

In [78]:
payment_methods = retrieve_all_payment_methods(customer_id)
payment_methods

[]

### Payment methods - Add

##### Function definition

In [59]:
def add_payment_method(customer_id, payment_type, payment_details):
    """
    Add a payment method (card/bank_account) to a Stripe customer.
    :param customer_id: str - The Stripe customer ID.
    :param payment_type: str - Type of payment method ('card' or 'bank_account').
    :param payment_details: dict - The payment details.
    :return: str - ID of the added payment method or None if there was an error.
    """
    try:
        # Convert payment details to a token
        token = stripe.Token.create(
            payment_type, payment_details
        )
        
        # Add token as a payment method to the customer
        payment_method = stripe.PaymentMethod.attach(
            token.id,
            customer=customer_id
        )

        return payment_method.id
    except stripe.error.StripeError as e:
        raise Exception(f"Failed to add payment method: {e}")

##### Function call

In [1]:
card_details = {
    "number": "4242424242424242",
    "exp_month": 12,
    "exp_year": 2024,
    "cvc": "123"
}

card_details_2 = {
    "number": "5555555555554444",
    "exp_month": 11,
    "exp_year": 2025,
    "cvc": "123"
}

bank_account_details = {
    "country": "US",
    "currency": "usd",
    "account_holder_name": "John Doe",
    "account_holder_type": "individual",
    "routing_number": "110000000",
    "account_number": "000123456789"
}


In [2]:
import time
import stripe

stripe.api_key = "pk_test_51O21ZuEIx0QzrdrDLBFzYIFbhTejaCj4ibToyVsBoWK4p3XlrpUFiMjajn3qiaw8cNeFL0jjngMgVhx24ZtIeBCi00kP29Pcr6"

In [6]:
# create payment method for card

payment_method_id = stripe.PaymentMethod.create(
  type="card",
  card=card_details,
)
print(payment_method_id)

{
  "billing_details": {
    "address": {
      "city": null,
      "country": null,
      "line1": null,
      "line2": null,
      "postal_code": null,
      "state": null
    },
    "email": null,
    "name": null,
    "phone": null
  },
  "card": {
    "brand": "visa",
    "checks": {
      "address_line1_check": null,
      "address_postal_code_check": null,
      "cvc_check": null
    },
    "country": "US",
    "exp_month": 12,
    "exp_year": 2024,
    "funding": "credit",
    "generated_from": null,
    "last4": "4242",
    "networks": {
      "available": [
        "visa"
      ],
      "preferred": null
    },
    "three_d_secure_usage": {
      "supported": true
    },
    "wallet": null
  },
  "created": 1701664273,
  "customer": null,
  "id": "pm_1OJToLEIx0QzrdrDnxIlZjAr",
  "livemode": false,
  "object": "payment_method",
  "type": "card"
}


In [7]:
payment_method_id = stripe.PaymentMethod.create(
  type="card",
  card=card_details_2,
)
print(payment_method_id)

{
  "billing_details": {
    "address": {
      "city": null,
      "country": null,
      "line1": null,
      "line2": null,
      "postal_code": null,
      "state": null
    },
    "email": null,
    "name": null,
    "phone": null
  },
  "card": {
    "brand": "mastercard",
    "checks": {
      "address_line1_check": null,
      "address_postal_code_check": null,
      "cvc_check": null
    },
    "country": "US",
    "exp_month": 11,
    "exp_year": 2025,
    "funding": "credit",
    "generated_from": null,
    "last4": "4444",
    "networks": {
      "available": [
        "mastercard"
      ],
      "preferred": null
    },
    "three_d_secure_usage": {
      "supported": true
    },
    "wallet": null
  },
  "created": 1701664336,
  "customer": null,
  "id": "pm_1OJTpMEIx0QzrdrDGOLTVxlG",
  "livemode": false,
  "object": "payment_method",
  "type": "card"
}
